In [2]:
import numpy as np
import pandas as pd

import requests
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import base64


In [52]:
client_id = '7a2b21cebdb44bc282e6e415eaccf347'
client_secret = 'a276bff35be5492da4e8c8015ebcd5ae'

# Codifica las credenciales en base64
encoded = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')

headers = {
    'Authorization': 'Basic ' + encoded
}

body = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=body)
access_token = auth_response.json()['access_token']

### Obtener albunes mas nuevos

In [65]:
headers = {
    'Authorization': 'Bearer ' + access_token
}

url = 'https://api.spotify.com/v1/browse/new-releases?market=ES'

response = requests.get(url, headers=headers)
data = response.json()


artist_list = []

if data.get('albums') and data['albums'].get('items'):
    for album in data['albums']['items']:
        for artist in album['artists']:
         
            artist_info = {
                'artist_name': artist['name'],
                'album_name': album['name'],
                'release_date': album['release_date'],
                'total_tracks': album['total_tracks'],
                'available_markets': album['available_markets'],
                'type': album['type'],     
            }

            artist_list.append(artist_info)

df_artists = pd.DataFrame(artist_list)

df_artists.head()




,artist_name,album_name,release_date,total_tracks,available_markets,type
0,GULEED,Cuando Menos Lo Espera,2023-07-07,1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album
1,Morad,Cuando Menos Lo Espera,2023-07-07,1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album
2,Ana Mena,bellodrama,2023-03-24,15,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album
3,Myke Towers,LA VIDA ES UNA,2023-03-23,23,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album
4,Eladio Carrion,3MEN2 KBRN,2023-03-17,18,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album


In [70]:

headers = {
    'Authorization': 'Bearer ' + access_token
}
top_tracks_url = "https://api.spotify.com/v1/playlists/37i9dQZEVXbNFJfN1Vw8d9/tracks"  
response = requests.get(top_tracks_url, headers=headers)

if response.status_code == 200:
    tracks_data = response.json()
    
    if 'items' in tracks_data:
        artist_ids = [artist['id'] for item in tracks_data['items'] for artist in item['track']['artists']]
        artist_ids = list(set(artist_ids))


        genres_info = {}

       
        for artist_id in artist_ids:
            artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
            artist_response = requests.get(artist_url, headers=headers)
            
            if artist_response.status_code == 200:
                artist_data = artist_response.json()
                artist_genres = artist_data.get('genres', [])
                
                for genre in artist_genres:
                    if genre in genres_info:
                        genres_info[genre] += 1
                    else:
                        genres_info[genre] = 1
          

    genres_df = pd.DataFrame(list(genres_info.items()), columns=['Genero', 'Count']).sort_values(by='Count', ascending=False)
    print(genres_df.head())  

            Genero  Count
3      trap latino     34
4    urbano latino     33
1        reggaeton     16
6   urbano espanol     10
0  flamenco urbano      8


In [71]:
headers = {
    'Authorization': 'Bearer ' + access_token
}


playlist_id = '37i9dQZEVXbNFJfN1Vw8d9'  

artist_ids = []

url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?market=ES&fields=items(track(artists(id,name)))&limit=100"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    playlist_data = response.json()
    
    for item in playlist_data['items']:
        track_artists = item['track']['artists']
        for artist in track_artists:
            if artist['id'] not in artist_ids:  
                artist_ids.append(artist['id'])

artists_data = []

for artist_id in artist_ids:
    artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
    artist_response = requests.get(artist_url, headers=headers)
    
    if artist_response.status_code == 200:
        artist_info = artist_response.json()
        
        name = artist_info['name']
        genres = artist_info['genres']
        popularity = artist_info['popularity']
        followers = artist_info['followers']['total']
        
        artists_data.append({
            'ID': artist_id,
            'Name': name,
            'Genres': genres,
            'Popularity': popularity,
            'Followers': followers
        })
    else:
        print(f"Error al obtener datos del artista {artist_id}: {artist_response.status_code}")

artists_df = pd.DataFrame(artists_data)

print(artists_df)

                        ID            Name  \
0   4ixNZvQuuzCDxBsxO9zE6L           Gonzy   
1   2O8vbr4RYPpk6MRA4fio7u           SAIKO   
2   4SsVbpTthjScTS7U2hmr1X        Arcángel   
3   0FwnPHExlRRxEZPLAi5tmG        JC Reyes   
4   3EiLUeyEcA6fbRPSHkG5kb    De La Ghetto   
..                     ...             ...   
70  3lY9Fxceu60W1rbon7PkuF     Omar Montes   
71  3qndygm4Vx7bEW8tNyDKSY           Camin   
72  7fL1kRUb9Zv1xxgOmsuUPn   Moncho Chavea   
73  1OGwUf0F69wrVxXYW3e9Nl         Jthyago   
74  4LuezDRPeENhjIphng4u15  Samueliyo Baby   

                                               Genres  Popularity  Followers  
0                                     [urbano latino]          65      51640  
1        [trap latino, urbano espanol, urbano latino]          78     693490  
2   [latin hip hop, reggaeton, trap latino, urbano...          83   13456254  
3                                    [urbano espanol]          74     538420  
4   [reggaeton, reggaeton flow, trap latino, urban..